In [1]:
pwd()

'C:\\Users\\rp2815'

In [1]:
#Import libraries:
import pandas as pd
import numpy as np
from sklearn.svm import LinearSVC
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Performing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

train = pd.read_csv('F:\second_term\ADS\proj3\Train.csv')
test = pd.read_csv('F:\second_term\ADS\proj3\Test.csv')
target = 'y' #this is y
IDcol = 'ID'

D:\Anaconda\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\Anaconda\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#Functions for building the model
def modelfit(alg, dtrain, dtest, predictors, performCV=True, printFeatureImportance=True, cv_folds=7):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['y'])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    #dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    dtest_predictions = alg.predict(dtest[predictors])
    
    #Perform cross-validation:
    if performCV:
        cv_score = cross_validation.cross_val_score(alg, dtrain[predictors], dtrain['y'], cv=cv_folds, scoring='roc_auc')
    
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy(train): %.4g" % metrics.accuracy_score(dtrain['y'].values, dtrain_predictions))
    print ("Accuracy(test) : %.4g" % metrics.accuracy_score(dtest['y'].values, dtest_predictions))
    #print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['y'], dtrain_predprob))
    
    if performCV:
        print ("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
        

In [43]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
#initial estimates:
#min_samples_split = 200 : This should be ~0.5-1% of total values.
#min_samples_leaf = 50 : Can be selected based on intuition. This is just used for preventing overfitting and again a small value because of imbalanced classes.
#max_depth = 1 : for decision stump
#max_features = ‘sqrt’ : Its a general thumb-rule to start with square root.
#subsample = 0.8 : This is a commonly-used start value
#n_estimators=100 : Can be selected based on intuition.
#max_features= "sqrt": typical sqrt to 30-40% of total features

#This is the untuned baseline, the parameters are not necesarily the best.
rf0 = LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.001, 
                C=30.0, multi_class='ovr', 
                fit_intercept=True, intercept_scaling=1, class_weight=None, 
                verbose=0, random_state=10, max_iter=250)
modelfit(rf0, train,test, predictors)#train:79% test:79%


Model Report
Accuracy(train): 0.7887
Accuracy(test) : 0.7925
CV Score : Mean - 0.8542047 | Std - 0.01755799 | Min - 0.833872 | Max - 0.8829901


In [ ]:
#Tuned parameter
#LinearSVC(penalty='l2', loss='squared_hinge', dual=True, tol=0.001, 
#                C=30.0, multi_class='ovr', 
#                fit_intercept=True, intercept_scaling=1, class_weight=None, 
#                verbose=0, random_state=10, max_iter=250)